In [ ]:
#CHECKPOINT 0: Create GCP instance w/ GPU and nice CPUs, fork test-gcp-transfer. 

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import shutil
shutil.unpack_archive("RealData.zip")

In [2]:
from Model import Model
from Losses import Losses
from ModelEvaluation import ModelEvaluation
from matplotlib import pyplot as plt
import tensorflow as tf
import numpy as np
import scipy.io
import numpy as np
from DataGenerator import DataGenerator
from TrainingManager import TrainingManager
from VideoGenerator import VideoGenerator


In [3]:
generator = VideoGenerator(topNm = 2000)


In [4]:
images, cleanImages, maps = generator.generateVideos(2)

  0%|          | 0/2 [00:00<?, ?it/s]

status 0


  0%|          | 0/2 [00:00<?, ?it/s]


error: OpenCV(4.6.0) /io/opencv/modules/imgproc/src/color.simd_helpers.hpp:94: error: (-2:Unspecified error) in function 'cv::impl::{anonymous}::CvtHelper<VScn, VDcn, VDepth, sizePolicy>::CvtHelper(cv::InputArray, cv::OutputArray, int) [with VScn = cv::impl::{anonymous}::Set<3, 4>; VDcn = cv::impl::{anonymous}::Set<3, 4>; VDepth = cv::impl::{anonymous}::Set<0, 2, 5>; cv::impl::{anonymous}::SizePolicy sizePolicy = cv::impl::<unnamed>::NONE; cv::InputArray = const cv::_InputArray&; cv::OutputArray = const cv::_OutputArray&]'
> Unsupported depth of input image:
>     'VDepth::contains(depth)'
> where
>     'depth' is 6 (CV_64F)


In [ ]:
def auxSlice(content, channels):
    numVid = content.shape[0]
    numFrame = content.shape[1]
    width = content.shape[2]
    height = content.shape[3]
    
    
    slices = np.zeros((numVid, numFrame*width, height, channels))
        
    for i in range(numVid):
        stack = []
        for k in range(numFrame):
            stack.append(content[i][k].reshape(width, height, channels))
        slices[i] = np.vstack(stack)
        
        
    return slices

        

In [ ]:
from matplotlib import pyplot as plt
import heapq
import csv
from matplotlib import cm

def showImage(image):
    data = np.reshape(image, (image.shape[0],image.shape[1],3))
    plt.imshow(imRegulate(data), interpolation='nearest')
    return plt.show()
    
    
def showMap(depthMap):

    d1 = depthMap.shape[0]
    d2 = depthMap.shape[1]
    
    pData = np.reshape(depthMap, (d1,d2))

    fig = plt.figure(figsize = (5,5))
    ax1 = fig.add_subplot(projection='3d')

    x = np.arange(0, d1)
    y = np.arange(0, d2)
    
    X, Y = np.meshgrid(x, y)
    surf = ax1.plot_surface(X, Y, pData, cmap = cm.coolwarm)

    ax1.set_zlabel("Thickness" + " (" + "nm" + ")")
    ax1.set_title('Patient Tear Film Thickness Profile')



    return plt.show()
    
        
def imRegulate(data):
    m = np.max(data)
    mi = np.min(data)
    norm = ((data - mi) / (m - mi))*255
    return norm.astype(np.uint8)


def tRegulate(data):
    data = tf.truediv(
       tf.subtract(
          data, 
          tf.reduce_min(data)
       ), 
       tf.subtract(
          tf.reduce_max(data), 
          tf.reduce_min(data)
       )
    )
    return tf.math.multiply(data,5000)

@tf.custom_gradient
def tCycle(depth):
    depth = tRegulate(depth)
    depth = tf.convert_to_tensor(depth)
    depth = tf.cast(tf.reshape(depth, (48*48)), tf.int32).numpy()
    print(depth)

    def grad(upstream):
            values = dColorMap[[depth]]
            ret = tf.convert_to_tensor(values.reshape(1,48,48,3).astype(np.float32))
            return ret * upstream
    
    values = colorMap[[depth]]
    return tf.convert_to_tensor(values.reshape(1,48,48,3).astype(np.float32)), grad


In [ ]:
import os
path = "/opt/conda/lib/python3.7/site-packages/models/images/"
counter = 1
for f in os.listdir(path):
    suffix = f.split('.')[-1]
    if suffix == 'jpg' or suffix == 'png':
        new = '{}.{}'.format(str(counter), suffix)
        os.rename(path + f, path + new)
        counter = int(counter) + 1

In [ ]:
losses = Losses()

In [ ]:
database = Database() #CHECKPOINT 1: Compile to this point.

In [ ]:
noisyRGB = {
    "gen_down" : [512,1024,2048],
    "gen_up": [512,1024,2048],
    "gen_dropout" : [True, False, False],
    "disc_down": [32, 64, 128, 256],
    "gen_lr": 0.002,
    "gen_b1": 0.5,
    "disc_lr" : 0.002,
    "disc_b1": 0.5,
    "epochs": 50,
    "time": database.fetchDate(),
    "name" : "longtest10/",
    "hsi" : False,
    "numPerlin": 5000,
    "numGaussian": 20000,
    "topNm": 2000,
    "notes": "Noisy, 0.1",
    "optimalCheckpoint": 10,
    "std": 0.1,
    "d1": 1728,
    "d2": 144
}

In [ ]:
vidRGB = {
    "gen_down" : [128,256,512],
    "gen_up": [128,256,512],
    "gen_dropout" : [True, True, False],
    "disc_down": [32],
    "gen_lr": 0.0002,
    "gen_b1": 0.5,
    "disc_lr" : 0.0002,
    "disc_b1": 0.5,
    "epochs": 6,
    "time": database.fetchDate(),
    "name" : "longtest1",
    "topNm": 5000,
    "notes": "Noisy, 0.1",
    "optimalCheckpoint": 10,
    "numVid": 10, 
    "d1": 1728, 
    "d2": 144
}

In [ ]:
noisyRGBManager = TrainingManager(noisyRGB, auxSlice(images, 3), auxSlice(maps, 1), auxSlice(images, 3) ,auxSlice(maps, 1))

In [ ]:
noisyRGBManager.previewGenerator()

In [ ]:
noisyRGBManager.previewDiscriminator()

In [ ]:
noisyRGBManager.fit() #CHECKPOINT 2: Finish a training session. 

In [ ]:
#CHECKPOINT 3: Create some testing infrastructure. --> Run some testing videos, get output, and display output. (One at a time for now)